In [ ]:
%pip install pandas numpy

In [2]:
import pandas as pd
import glob
import json

# Path to JSON files
#LISTINGS_PATH = "/Users/vivekprabu/Downloads/nano-slack-master/ShopTalk/data/*.json"
LISTINGS_PATH = "/Users/mitilroy/IK/Projects/Capstone/ShopTalk/abo-listings/listings/metadata/*.json"
json_files = glob.glob(LISTINGS_PATH)

dfs = []

for f in json_files:
    with open(f, 'r') as file:
        for line in file:  # Read line by line to handle multiple JSON objects
            data = json.loads(line.strip())  # Parse each JSON object separately
            
            extracted_data = {}  # Store extracted values
            
            # Loop through each key in the JSON object
            for key, value in data.items():
                if isinstance(value, list):  # Only process lists
                    for item in value:
                        if isinstance(item, dict) and "language_tag" in item and item["language_tag"].startswith("en_"):
                            extracted_data[key] = item["value"]  # Store the corresponding value
                        elif isinstance(item, dict) and "language_tag" not in item and "value" in item:
                            extracted_data[key] = item["value"]
                else:
                    extracted_data[key] = value
            
            # Convert extracted data into DataFrame
            df = pd.DataFrame([extracted_data])
            dfs.append(df)

# Combine all DataFrames
df_products = pd.concat(dfs, ignore_index=True)

# Display result
df_products.head()

,brand,bullet_point,color,item_id,item_name,item_weight,model_name,model_number,product_type,main_image_id,...,style,item_dimensions,fabric_type,product_description,item_shape,pattern,finish_type,spin_id,model_year,3dmodel_id
0,Amazon Brand - Solimo,None,Others,B07T6RZ2CM,Amazon Brand - Solimo Designer Couples Sitting...,50.0,Lenovo K4 Note,gz8115-SL40423,CELLULAR_PHONE_CASE,71dZhpsferL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Amazon Brand - Solimo,No Warranty,others,B07T2JY31Y,Amazon Brand - Solimo Designer Leaf on Wood 3D...,50.0,Sony Xperia Z1 L39H,gz8056-SL40528,CELLULAR_PHONE_CASE,71vX7qIEAIL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Stone & Beam,NaN,NaN,B0849YGSCZ,Stone & Beam Contemporary Doily Wool Farmhouse...,NaN,NaN,I59I8044IVYGRYC00-Parent,HOME_FURNITURE_AND_DECOR,A1EZF-2mB5L,...,Contemporary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Amazon Brand - Solimo,"Color: Multicolor including green, blue, and p...",Multicolor,B081K6TCML,Amazon Brand - Solimo Plastic Multipurpose Mod...,2850.0,NaN,sol_cujo_13,HOME,81o9EyZ-fAL,...,NaN,{'height': {'normalized_value': {'unit': 'inch...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Amazon Brand - Solimo,Extreme precision design allows easy access to...,Multicolor,B0854774X5,Amazon Brand - Solimo Designer Take It Easy UV...,50.0,Nokia 8.1,UV10714-SL40617,CELLULAR_PHONE_CASE,81xaJCVnl3L,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Rename main_image_id to image_id
df_products.rename(columns={"main_image_id": "image_id"}, inplace=True)

In [6]:
#IMAGES_PATH = "/Users/vivekprabu/Downloads/nano-slack-master/ShopTalk/data/images.csv"
IMAGES_PATH = "/Users/mitilroy/IK/Projects/Capstone/ShopTalk/abo-images-small/images/metadata/images.csv"

df_images = pd.read_csv(IMAGES_PATH)
df_images.head()

,image_id,height,width,path
0,010-mllS7JL,106,106,14/14fe8812.jpg
1,01dkn0Gyx0L,122,122,da/daab0cad.jpg
2,01sUPg0387L,111,111,d2/d2daaae9.jpg
3,1168jc-5r1L,186,186,3a/3a4e88e6.jpg
4,11RUV5Fs65L,30,500,d9/d91ab9cf.jpg


In [7]:
# rename path to image_path
df_images.rename(columns={"path": "image_path"}, inplace=True)

In [8]:
df_product_image_merged = pd.merge(df_products, df_images, on="image_id", how="outer")
df_product_image_merged.head()

,brand,bullet_point,color,item_id,item_name,item_weight,model_name,model_number,product_type,image_id,...,product_description,item_shape,pattern,finish_type,spin_id,model_year,3dmodel_id,height,width,image_path
0,NaN,NaN,NaN,B07Y7M8LV7,NaN,NaN,NaN,NaN,KITCHEN,010-mllS7JL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106.0,106.0,14/14fe8812.jpg
1,NaN,NaN,NaN,B07Y7M4JZ2,NaN,NaN,NaN,NaN,KITCHEN,01dkn0Gyx0L,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,122.0,122.0,da/daab0cad.jpg
2,NaN,NaN,NaN,B07ZTFRVTT,NaN,NaN,NaN,NaN,KITCHEN,01sUPg0387L,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,111.0,111.0,d2/d2daaae9.jpg
3,NaN,NaN,NaN,B07ZTFC41S,NaN,NaN,NaN,NaN,KITCHEN,1168jc-5r1L,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,186.0,186.0,3a/3a4e88e6.jpg
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11RUV5Fs65L,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.0,500.0,d9/d91ab9cf.jpg


In [9]:
df_product_image_merged.shape

(422403, 28)

In [10]:
df_product_image_merged.isna().sum()

brand                  303815
bullet_point           308922
color                  323885
item_id                274701
item_name              299669
item_weight            316209
model_name             347729
model_number           298312
product_type           274701
image_id                  575
item_keywords          314835
country                274701
marketplace            274701
domain_name            274701
material               375463
style                  391998
item_dimensions        379065
fabric_type            416850
product_description    420398
item_shape             418460
pattern                418928
finish_type            421048
spin_id                414181
model_year             414638
3dmodel_id             414443
height                    575
width                     575
image_path                575
dtype: int64

In [11]:
## of this Item ID is the most important identifier - we cannot retrieve any product without item id. So we will drop anything with Item ID as null
df_merged_non_null_item_id = df_product_image_merged.dropna(subset=['item_id'])
print(df_merged_non_null_item_id.shape)
df_merged_non_null_item_id.head()

(147702, 28)


,brand,bullet_point,color,item_id,item_name,item_weight,model_name,model_number,product_type,image_id,...,product_description,item_shape,pattern,finish_type,spin_id,model_year,3dmodel_id,height,width,image_path
0,NaN,NaN,NaN,B07Y7M8LV7,NaN,NaN,NaN,NaN,KITCHEN,010-mllS7JL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106.0,106.0,14/14fe8812.jpg
1,NaN,NaN,NaN,B07Y7M4JZ2,NaN,NaN,NaN,NaN,KITCHEN,01dkn0Gyx0L,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,122.0,122.0,da/daab0cad.jpg
2,NaN,NaN,NaN,B07ZTFRVTT,NaN,NaN,NaN,NaN,KITCHEN,01sUPg0387L,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,111.0,111.0,d2/d2daaae9.jpg
3,NaN,NaN,NaN,B07ZTFC41S,NaN,NaN,NaN,NaN,KITCHEN,1168jc-5r1L,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,186.0,186.0,3a/3a4e88e6.jpg
6,NaN,NaN,NaN,B07ZT8VS6F,NaN,NaN,NaN,NaN,AV_FURNITURE,11Y+Xpt1lfL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,103.0,196.0,99/9987a1c8.jpg


In [12]:
df_merged_non_null_item_id.isna().sum()

brand                   29114
bullet_point            34221
color                   49184
item_id                     0
item_name               24968
item_weight             41508
model_name              73028
model_number            23611
product_type                0
image_id                  575
item_keywords           40134
country                     0
marketplace                 0
domain_name                 0
material               100762
style                  117297
item_dimensions        104364
fabric_type            142149
product_description    145697
item_shape             143759
pattern                144227
finish_type            146347
spin_id                139480
model_year             139937
3dmodel_id             139742
height                    575
width                     575
image_path                575
dtype: int64

In [18]:
df_merged = df_merged_non_null_item_id.copy()
df_merged["enhanced_product_desc"] = df_merged["product_description"].fillna("").astype(str) \
        + " " + df_merged["bullet_point"].fillna("").astype(str) \
        + ", brand: " + df_merged["brand"].fillna("").astype(str) \
        + ", weight: " + df_merged["item_weight"].fillna("").astype(str) \
        + ", color: " + df_merged["color"].fillna("").astype(str) \
        + ", height: " + df_merged["height"].fillna("").astype(str) \
        + ", width: " + df_merged["width"].fillna("").astype(str) \
        + ", model year: " + df_merged["model_year"].fillna("").astype(str) \
        + ", shape: " + df_merged["item_shape"].fillna("").astype(str) \
        + ", style: " + df_merged["style"].fillna("").astype(str) \
        + ", material: " + df_merged["material"].fillna("") \
        + ", product_type: " + df_merged["product_type"].fillna("").astype(str)
df_merged.head()

,brand,bullet_point,color,item_id,item_name,item_weight,model_name,model_number,product_type,image_id,...,item_shape,pattern,finish_type,spin_id,model_year,3dmodel_id,height,width,image_path,enhanced_product_desc
0,NaN,NaN,NaN,B07Y7M8LV7,NaN,NaN,NaN,NaN,KITCHEN,010-mllS7JL,...,NaN,NaN,NaN,NaN,NaN,NaN,106.0,106.0,14/14fe8812.jpg,", brand: , weight: , color: , height: 106.0, ..."
1,NaN,NaN,NaN,B07Y7M4JZ2,NaN,NaN,NaN,NaN,KITCHEN,01dkn0Gyx0L,...,NaN,NaN,NaN,NaN,NaN,NaN,122.0,122.0,da/daab0cad.jpg,", brand: , weight: , color: , height: 122.0, ..."
2,NaN,NaN,NaN,B07ZTFRVTT,NaN,NaN,NaN,NaN,KITCHEN,01sUPg0387L,...,NaN,NaN,NaN,NaN,NaN,NaN,111.0,111.0,d2/d2daaae9.jpg,", brand: , weight: , color: , height: 111.0, ..."
3,NaN,NaN,NaN,B07ZTFC41S,NaN,NaN,NaN,NaN,KITCHEN,1168jc-5r1L,...,NaN,NaN,NaN,NaN,NaN,NaN,186.0,186.0,3a/3a4e88e6.jpg,", brand: , weight: , color: , height: 186.0, ..."
6,NaN,NaN,NaN,B07ZT8VS6F,NaN,NaN,NaN,NaN,AV_FURNITURE,11Y+Xpt1lfL,...,NaN,NaN,NaN,NaN,NaN,NaN,103.0,196.0,99/9987a1c8.jpg,", brand: , weight: , color: , height: 103.0, ..."


In [19]:
df_merged.isna().sum()

brand                     29114
bullet_point              34221
color                     49184
item_id                       0
item_name                 24968
item_weight               41508
model_name                73028
model_number              23611
product_type                  0
image_id                    575
item_keywords             40134
country                       0
marketplace                   0
domain_name                   0
material                 100762
style                    117297
item_dimensions          104364
fabric_type              142149
product_description      145697
item_shape               143759
pattern                  144227
finish_type              146347
spin_id                  139480
model_year               139937
3dmodel_id               139742
height                      575
width                       575
image_path                  575
enhanced_product_desc         0
dtype: int64

In [21]:
df_merged_master = df_merged.copy()
selected_columns = ['item_id', 'item_name', 'enhanced_product_desc', 'image_path']
df_merged_master = df_merged_master[selected_columns]
## Drop duplicates
df_merged_master.drop_duplicates()
print("## Master DF with null values")
print(df_merged_master.isna().sum())
print("## Shape of master dataframe", df_merged_master.shape)
df_merged_master.head()

## Master DF with null values
item_id                      0
item_name                24968
enhanced_product_desc        0
image_path                 575
dtype: int64
## Shape of master dataframe (147702, 4)


,item_id,item_name,enhanced_product_desc,image_path
0,B07Y7M8LV7,NaN,", brand: , weight: , color: , height: 106.0, ...",14/14fe8812.jpg
1,B07Y7M4JZ2,NaN,", brand: , weight: , color: , height: 122.0, ...",da/daab0cad.jpg
2,B07ZTFRVTT,NaN,", brand: , weight: , color: , height: 111.0, ...",d2/d2daaae9.jpg
3,B07ZTFC41S,NaN,", brand: , weight: , color: , height: 186.0, ...",3a/3a4e88e6.jpg
6,B07ZT8VS6F,NaN,", brand: , weight: , color: , height: 103.0, ...",99/9987a1c8.jpg


In [22]:
## Dropping null item name rows - as the product recommendation should give a product name
df_merged_master.dropna(subset=['item_name'], inplace=True)
print("## Master DF with null values")
print(df_merged_master.isna().sum())
print("## Shape of master dataframe", df_merged_master.shape)
df_merged_master.head()

## Master DF with null values
item_id                    0
item_name                  0
enhanced_product_desc      0
image_path               460
dtype: int64
## Shape of master dataframe (122734, 4)


,item_id,item_name,enhanced_product_desc,image_path
8,B08G1FC33N,UMI by Amazon Modern Wood Cat Tree Cats Multi ...,Easy to install: With a clear installation fo...,ee/ee239f0f.jpg
658,B07C74RX1X,AmazonBasics DisplayPort to HDMI A/M cable - 3...,From DisplayPort to HDMI only (not bi-directi...,1e/1e00bd6c.jpg
674,B07W95HYLH,"Happy Belly Water, Seltzer, 33.8 Ounce","An Amazon brand, brand: Happy Belly, weight: ...",36/36b8f278.jpg
675,B084NV7PD9,Lemon Macaron 3ct,", brand: Fresh, weight: , color: , height: 40...",36/36b8f278.jpg
676,B084Q3PD5R,Jumbo Peanut Butter Cookie 12ct,", brand: Fresh, weight: , color: , height: 40...",36/36b8f278.jpg


In [23]:
## This set is good - as we dont necessarily have to have an image in the product recommendation.
## We will start working with 20000 recs. As having 122734 recs to be processed, it will greatly increase the time required for processing

df_sample = df_merged_master.sample(20000)
df_sample.reset_index(drop=True, inplace=True)

print("## Shape of sample dataframe", df_sample.shape)
df_sample.head()

## Shape of sample dataframe (20000, 4)


,item_id,item_name,enhanced_product_desc,image_path
0,B07NC41NCW,"Amazon Brand - find. Women's Chelsea Boots, Br...","Outer: Leather, brand: find., weight: , color...",45/45f10a30.jpg
1,B07TG4KYZC,Amazon Brand - Solimo Designer Multicolor Spar...,"No Warranty, brand: Amazon Brand - Solimo, we...",aa/aabdf384.jpg
2,B07TCWT4NT,Amazon Brand - Solimo Designer Eiffel Tower 3D...,"No Warranty, brand: Amazon Brand - Solimo, we...",3a/3aaed179.jpg
3,B008BS2678,Sterling Silver Hoop Earrings set with Swarovs...,"Imported, brand: Amazon Collection, weight: ,...",b9/b98c307a.jpg
4,B07TG4V82Y,Amazon Brand - Solimo Designer Lion with Lione...,"None, brand: Amazon Brand - Solimo, weight: 5...",01/01631c37.jpg
